# Quantum Traveling Salesman Problem Solver

This notebook implements a quantum solution for the Traveling Salesman Problem (TSP) using Qiskit.
The TSP is about finding the shortest possible route that visits each city exactly once and returns to the origin city.

We'll use the Quantum Approximate Optimization Algorithm (QAOA) approach.

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
import networkx as nx
import matplotlib.pyplot as plt

## Problem Setup

First, we'll create a small example problem with 4 cities.
The distances between cities are represented using an adjacency matrix.

In [ ]:
# Define cities and their distances
number_of_cities = 4

# Create a random distance matrix (symmetric)
# distances[i][j] represents the distance between city i and city j
np.random.seed(42)  # For reproducibility
distances = np.random.rand(number_of_cities, number_of_cities)
distances = (distances + distances.T) / 2  # Make symmetric
np.fill_diagonal(distances, 0)  # Distance to same city is 0

# Visualize the cities as a graph
graph = nx.from_numpy_array(distances)
pos = nx.spring_layout(graph)
nx.draw(graph, pos, with_labels=True)
labels = nx.get_edge_attributes(graph, 'weight')
nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
plt.show()

## Quantum Circuit Implementation

We'll encode the TSP into a Quantum Approximate Optimization Algorithm (QAOA) problem.
The encoding uses binary variables x_{i,p} where:
- i represents the city
- p represents the position in the tour

Constraints:
1. Each city must be visited exactly once
2. Each position must be used exactly once
3. The total distance should be minimized

In [ ]:
def create_tsp_quantum_circuit(distances, number_of_qubits):
    """
    Creates a quantum circuit for solving the TSP problem.
    
    Args:
        distances (np.array): Matrix of distances between cities
        number_of_qubits (int): Number of qubits needed for encoding
        
    Returns:
        QuantumCircuit: Circuit implementing the TSP problem
    """
    # Create quantum circuit with required number of qubits
    quantum_circuit = QuantumCircuit(number_of_qubits)
    
    # Create initial superposition
    for qubit in range(number_of_qubits):
        quantum_circuit.h(qubit)
    
    # Add problem constraints (simplified version)
    # In a full implementation, we would add more gates to enforce:
    # - One-hot encoding for city positions
    # - Valid tour constraints
    # - Distance minimization objective
    
    return quantum_circuit

# Calculate required number of qubits
# We need log2(number_of_cities!) qubits for a full implementation
# For simplicity, we'll use number_of_cities^2 qubits here
number_of_qubits = number_of_cities * number_of_cities

# Create quantum circuit
quantum_circuit = create_tsp_quantum_circuit(distances, number_of_qubits)

# Set up the quantum instance using Aer's QASM simulator
quantum_backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(quantum_backend, shots=1000)

# Display the circuit
print("Quantum Circuit for TSP:")
quantum_circuit.draw()

## Execute the Quantum Circuit

Now we'll run the circuit and analyze the results.
Note: This is a simplified implementation. A full TSP solution would require:
1. More sophisticated qubit encoding
2. Additional constraints for valid tours
3. Problem-specific mixing operators
4. Post-processing to ensure valid solutions

In [ ]:
# Execute the circuit
job = execute(quantum_circuit, quantum_backend, shots=1000)
result = job.result()

# Get the counts of measurement results
counts = result.get_counts(quantum_circuit)

# Post-process the results
def interpret_bitstring(bitstring, number_of_cities):
    """
    Convert a measurement bitstring into a possible tour.
    This is a simplified interpretation - a full implementation would need more sophisticated decoding.
    """
    # Reshape the bitstring into a matrix
    bits = [int(bit) for bit in bitstring]
    matrix = np.array(bits).reshape(number_of_cities, number_of_cities)
    
    # Try to extract a valid tour (simplified)
    tour = []
    for position in range(number_of_cities):
        city_bits = matrix[:, position]
        if 1 in city_bits:
            tour.append(np.where(city_bits == 1)[0][0])
    return tour if len(tour) == number_of_cities else None

# Analyze results
print("\nPossible Tours:")
valid_tours = 0
for bitstring, count in counts.items():
    tour = interpret_bitstring(bitstring, number_of_cities)
    if tour is not None:
        # Calculate tour distance
        tour_distance = sum(distances[tour[i]][tour[(i+1)%number_of_cities]] 
                          for i in range(number_of_cities))
        print(f"Tour {tour}: Distance {tour_distance:.2f} (Found {count} times)")
        valid_tours += count

print(f"\nFound {valid_tours} valid tours out of 1000 measurements")

# Note: Due to the simplified implementation, many measurements
# might not correspond to valid tours. A full implementation would
# use additional quantum gates and constraints to ensure valid solutions.